In [49]:
from py2neo import Graph, Node, Relationship

In [50]:
g=Graph("bolt://localhost:7687",password="yourpassword")

In [51]:
g.run("MATCH (n) DETACH DELETE n")

# INSERTIONS

### LOADING MOVIES

In [52]:
loadMovies = '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///moviesdesc.csv" AS row
MERGE(m:Movie{title:row.Title, runtime:row.Runtime})
MERGE(c:Company{name:row.Company})
MERGE (m) -[:producedBy]-> (c) 
MERGE(p:Country{name:row.Country})
MERGE (m) -[:filmedIn]-> (p) 
MERGE(f1:Filmer{name:row.Director})
MERGE (m) -[:directedBy]-> (f1) 
MERGE(f2:Filmer{name:row.Writer})
MERGE (m) -[:writtenBy]-> (f2) 
MERGE(g:Genre{name:row.Genre})
MERGE (m) -[:hasGenre]-> (g)
MERGE(y:Year{year:toInt(row.Year)})
MERGE (m) -[:filmedYear]-> (y)

'''

In [53]:
g.run(loadMovies)

### LOADING USERS

In [54]:
loadUsers = '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///usermovies2.csv" AS row
MERGE(r:Reviewer{ID:row.Userid})
MERGE(a:Age{age:toInt(row.AgeGrader)})
MERGE (r) -[:hasAge]-> (a)
MERGE(g:Gender{gender:row.GenderGrader})
MERGE (r) -[:hasGender]-> (g)
MERGE(o:Occupation{name:row.OccupationGrader})
MERGE (r) -[:hasOccupation]-> (o)
'''

In [55]:
g.run(loadUsers)

### LOADING RATINGS (Match between Movie and Reviewer rating)

In [56]:
loadRatings = '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///usermovies2.csv" AS row
MATCH (m:Movie{title:row.MovieTitle})-[:filmedYear] -> (y:Year{year: toInt(row.ReleaseDate)})
MATCH (r:Reviewer{ID:row.Userid})
MERGE (r) -[:rates{rating:toInt(row.RatingGrader)}]-> (m)
'''

In [57]:
g.run(loadRatings)

### ADD AVGRating AND NRatings FOR REVIEWER

In [58]:
addAVGReviewer = '''
MATCH (rev:Reviewer)-[r:rates]->()
WITH rev,count(r) as nRatings, avg(r.rating) as avgRating
ORDER BY avgRating DESC 
SET rev.avgRating = avgRating, rev.nRatings = nRatings
RETURN rev,nRatings, avgRating
'''

In [59]:
g.run(addAVGReviewer)

### ADD AVGRating AND NRatings for Movie

In [60]:
addAVGMovie = '''
MATCH ()-[r:rates]->(m:Movie) 
WITH m,count(r) as nRatings, avg(r.rating) as avgRating 
ORDER BY avgRating DESC
SET m.avgRating = avgRating, m.nRatings = nRatings
RETURN m,nRatings, avgRating
'''

In [61]:
g.run(addAVGMovie)

### ADD denominator of cosine distance to the reviewers

In [ ]:
addCosine = '''
MATCH ()-[r:rates]->(m:Movie) 
WITH m,count(r) as nRatings, avg(r.rating) as avgRating 
ORDER BY avgRating DESC
SET m.avgRating = avgRating, m.nRatings = nRatings
RETURN m,nRatings, avgRating
'''

In [ ]:
g.run(addCosine)

# QUERIES

### CHECKING MOVIES THAT MATCH

In [ ]:
checkMatchMovies = '''
MATCH ()-[r:rates]->(m:Movie) 
WITH m,count(r) as nRatings 
ORDER BY nRatings DESC 
RETURN m,nRatings
'''

In [ ]:
g.run(checkMatchMovies)

### TOTAL MOVIES

In [ ]:
nMovies = '''
MATCH (m:Movie) RETURN count(m)
'''

In [ ]:
g.run(nMovies)

### TOTAL RATES

In [ ]:
nRates = '''
MATCH ()-[r:rates]->()
RETURN count(r)
'''

In [ ]:
g.run(nRates)

### NUMBER OF RATINGS BY REVIEWER

In [ ]:
checkReviewers = '''
MATCH (rev:Reviewer)-[r:rates]->() 
WITH rev,count(r) as nRatings 
ORDER BY nRatings DESC 
RETURN rev,nRatings
'''

In [ ]:
g.run(checkReviewers)

# RECOMMENDATIONS

### 20 BEST MOVIES

In [ ]:
findTop20 = '''
MATCH (r:Reviewer)-[ra:rates]->(m:Movie)
WHERE m.avgRating>4
RETURN m.title,count(ra) AS NumberofReviews, m.avgRating AS AverageRating, count(ra)*m.avgRating as score
ORDER BY score desc limit 20;
'''

In [ ]:
g.run(findTop20)

### FIND BETTER MOVIES ACCORDING TO THE MOST SIMILAR USERS

In [ ]:
findSimilarUsers = '''
MATCH (r:Reviewer{ID:"999"})-[v:rates]->(m:Movie)
WITH r,m,v.rating -r.avgRating as rRates
MATCH (o:Reviewer)-[w:rates]->(m)
WHERE NOT o.ID="999"
WITH r,m,rRates, o, w.rating - o.avgRating as oRates
WITH r,rRates,o,oRates,m, rRates*oRates as mult
WITH o,sum(mult)/(r.squareRates*o.squareRates) as cos
ORDER BY cos DESC LIMIT 25
MATCH (m1:Movie) 
WHERE NOT (:Reviewer{ID:"999"})-[:rates]->(m1:Movie) AND m1.nRatings > 0
MATCH (o:Reviewer)-[x:rates]->(m1)
WITH m1, avg(x.rating) as suggestedRating, count(x.rating) as nORating, (avg(x.rating)*0.8 + (1.0*count(x.rating)/25)*5*0.2) as score
ORDER BY score DESC
WHERE nORating >= 3 AND suggestedRating >= 3
RETURN m1, suggestedRating, nORating, score
'''

In [ ]:
g.run(findSimilarUsers)

### FIND 100 MOST SIMILAR MOVIES TO THE LATEST ONE SEEN

In [ ]:
findSimilarMovies = '''
MATCH (y1:Year)<-[:filmedYear]-(m1:Movie{title:'Aliens'})-[:hasGenre]->(g1:Genre)
MATCH (w1:Filmer)<-[:writtenBy]-(m1)-[:directedBy]->(d1:Filmer)
MATCH (c1:Company)<-[:producedBy]-(m1)-[:filmedIn]->(p1:Country)
MATCH (y2:Year)<-[:filmedYear]-(m2:Movie)-[:hasGenre]->(g2:Genre) WHERE m1 <> m2
MATCH (w2:Filmer)<-[:writtenBy]-(m2)-[:directedBy]->(d2:Filmer)
MATCH (c2:Company)<-[:producedBy]-(m2)-[:filmedIn]->(p2:Country)
WITH m1,g1,m2,g2, (CASE WHEN g1 = g2 THEN 1 ELSE 0 END)*0.4 AS g, (CASE WHEN d1 = d2 THEN 1 ELSE 0 END)*0.2 AS d, 
(CASE WHEN w1 = w2 THEN 1 ELSE 0 END)*0.1 AS w, (CASE WHEN c1 = c2 THEN 1 ELSE 0 END)*0.1 AS c, 
(CASE WHEN p1 = p2 THEN 1 ELSE 0 END)*0.1 AS p, (CASE WHEN abs(y1.year-y2.year) > 10 THEN 0.0 ELSE 10-abs(y1.year-y2.year) END)*0.01 AS y,
(CASE WHEN m2.avgRating IS NULL OR m2.nRatings < 5 THEN 0 ELSE m2.avgRating END)/5 AS avgRat 
RETURN m2,g2,g+d+w+c+p+y+ avgRat as score1
ORDER BY score1 DESC
LIMIT 100
'''

In [ ]:
g.run(findSimilarMovies)